# 002 JOIN

```SQL
-- Limpieza de tablas previas para evitar conflictos
DROP TABLE IF EXISTS stock;
DROP TABLE IF EXISTS ventas_dia;
DROP TABLE IF EXISTS productos;

-- 1. Tabla de Catálogo de Productos
CREATE TABLE productos (
    producto_id INTEGER PRIMARY KEY,
    nombre_producto VARCHAR(50),
    categoria VARCHAR(50)
);

-- 2. Tabla de Ventas Diarias
CREATE TABLE ventas_dia (
    venta_id INTEGER PRIMARY KEY,
    producto_id INTEGER, -- No ponemos Foreign Key para poder insertar el ID 99 (error)
    cantidad INTEGER
);

-- 3. Tabla de Stock en Bodega
CREATE TABLE stock (
    producto_id INTEGER,
    cantidad_disponible INTEGER
);

-- Insertar en Catálogo
INSERT INTO productos (producto_id, nombre_producto, categoria)
VALUES
    (10, 'Espresso', 'Bebida'),
    (20, 'Latte', 'Bebida'),
    (30, 'Muffin', 'Comida'),
    (40, 'Galleta', 'Comida');

-- Insertar en Ventas del Día
-- Nota el ID 99 que no existe en el catálogo
INSERT INTO ventas_dia (venta_id, producto_id, cantidad)
VALUES
    (501, 10, 5),
    (502, 20, 3),
    (503, 99, 1);

-- Insertar en Stock
-- Nota el ID 50 que es un producto nuevo que no está en el catálogo
INSERT INTO stock (producto_id, cantidad_disponible)
VALUES
    (10, 100),
    (20, 50),
    (30, 20),
    (50, 10);

SELECT * FROM productos;
SELECT * FROM ventas_dia;
SELECT * FROM stock;
```

In [3]:
import pandas as pd
import numpy as np

In [24]:
df_productos = pd.read_csv('Data/002/productos.csv')
df_stock = pd.read_csv('Data/002/stock.csv')
df_ventas = pd.read_csv('Data/002/ventas_dia.csv')

df_productos.head()

,producto_id,nombre_producto,categoria,precio
0,10,Espresso,Bebida,3.5
1,20,Latte,Bebida,4.5
2,30,Muffin,Comida,2.5
3,40,Galleta,Comida,2.0


In [3]:
df_stock.head()

,producto_id,cantidad_disponible
0,10,100
1,20,50
2,30,20
3,50,10


In [4]:
df_ventas.head()

,venta_id,producto_id,cantidad
0,501,10,5
1,502,20,3
2,503,99,1


# Reto 1 (Ventas Limpias): Coincidencia exacta entre productos y ventas_dia.

In [8]:
df_merge1 = df_ventas.merge(df_productos, on='producto_id', how='inner')

respuesta1 = df_merge1[['venta_id','nombre_producto']]

respuesta1

,venta_id,nombre_producto
0,501,Espresso
1,502,Latte


```SQL
SELECT
    v.venta_id,
    p.nombre_producto
FROM ventas_dia v
INNER JOIN productos p ON v.producto_id = p.producto_id
```

# Reto 2 (Auditoría Catálogo): Todos los productos, se hayan vendido o no.

In [14]:
df_merge2 = df_productos.merge(df_ventas, on='producto_id', how='left')

respuesta2 = df_merge2[['venta_id','nombre_producto']]

respuesta2

,venta_id,nombre_producto
0,501.0,Espresso
1,502.0,Latte
2,NaN,Muffin
3,NaN,Galleta


# Reto 3 (Alerta Errores): Solo los registros de ventas_dia que no tienen "padre" en el catálogo.

In [17]:
df_merge3 = df_ventas.merge(df_productos, on='producto_id', how='left')

respuesta3 = df_merge3[
    (df_merge3['nombre_producto'].isna())
]

respuesta3 = respuesta3[['venta_id','nombre_producto']]

respuesta3

,venta_id,nombre_producto
2,503,NaN


```SQL
SELECT
    v.venta_id,
    p.nombre_producto
FROM ventas_dia v
LEFT JOIN productos p ON p.producto_id = v.producto_id
WHERE p.nombre_producto IS NULL;
```

# Reto 4: "Queremos un reporte total para el inventariador. Debe mostrar todos los productos del catálogo y todos los productos que hay en stock.

In [20]:
df_merge4 = df_productos.merge(df_stock, on='producto_id', how='outer')

df_merge4['producto_id'] = df_merge4['producto_id'].fillna(df_merge4['producto_id'])

respuesta4 = df_merge4[['producto_id','nombre_producto','cantidad_disponible']]

respuesta4

,producto_id,nombre_producto,cantidad_disponible
0,10,Espresso,100.0
1,20,Latte,50.0
2,30,Muffin,20.0
3,40,Galleta,NaN
4,50,NaN,10.0


### ¿Cuánto dinero hemos recaudado en total por cada CATEGORÍA de producto (Bebida, Comida)? Solo incluye productos que tengan ventas.

In [25]:
df_merge5 = df_ventas.merge(df_productos, on='producto_id', how='inner')

df_merge5['total'] = df_merge5['cantidad'] * df_merge5['precio']

# Agrupamos por categoría y sumamos la columna 'total'
df_agrupado = df_merge5.groupby('categoria')['total'].sum().reset_index()

# Visualizamos el resultado
print(df_agrupado)

  categoria  total
0    Bebida   31.0


```SQL
SELECT
    p.categoria,
    SUM(v.cantidad * p.precio) AS total
FROM ventas_dia v
INNER JOIN productos p ON v.producto_id  = p.producto_id
GROUP BY p.categoria
```